In [1]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import v2 as transforms
from sklearn.metrics import accuracy_score,mean_absolute_error
from tqdm import tqdm
from torchvision import datasets, models
from PIL import Image
datasetPath = ".."
anotationsPath = "..\\dataset\\anotations"


# Load the annotations
with open(os.path.join(anotationsPath, "annotations.json")) as f:
    annotations = json.load(f)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Create labels


# Create Dataset

In [2]:
class CustomDataset(Dataset):
    def __init__(self, datasetPath, imageIds,annotations,ImgPaths, transform=None):
        self.datasetPath = datasetPath
        self.imageIds = imageIds
        self.labels = {}
        self.imagePaths = {annotation['id']: annotation['path'] for annotation in ImgPaths}
        self.transform = transform
        self.load_labels(annotations)
        self.images = {}
        for image_id in self.imageIds:
            image_path = os.path.join(self.datasetPath, self.imagePaths[image_id])
            img = Image.open(image_path).convert("RGB")
            if self.transform:
                img = self.transform(img)
            else:
                img = transforms.ToTensor()(img)
            self.images[image_id] = img
    def __len__(self):
        return len(self.imageIds)
    def load_labels(self,annotations):
        for annotation in annotations:
            label = annotation['image_id']
            if label not in self.imageIds:
                continue
            if label not in self.labels:
                self.labels[label] = 0
            else:
                self.labels[label] += 1
    def __getitem__(self, idx):
        #print(self.imageIds[idx])
        image_id =self.imageIds[idx]
        img =self.images[image_id]

        label = torch.tensor(self.labels[image_id], dtype=torch.float32)
        return img, label
#print(annotations['annotations']["pieces"])



# Training

In [3]:
def epoch_iter(dataloader, model, loss_fn, optimizer=None, is_train=True):
    if is_train:
      assert optimizer is not None, "When training, please provide an optimizer."
      
    num_batches = len(dataloader)

    if is_train:
      model.train() # put model in train mode
    else:
      model.eval()

    total_loss = 0.0
    preds = []
    labels = []

    with torch.set_grad_enabled(is_train):
      for batch, (X, y) in enumerate(tqdm(dataloader)):
          X, y = X.to(device), y.to(device)

          # Compute prediction error
          pred = model(X)
          pred = pred.squeeze()
          #print(pred.shape)
          #print(y.shape)
          loss = loss_fn(pred, y)

          if is_train:
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

          # Save training metrics
          total_loss += loss.item() # IMPORTANT: call .item() to obtain the value of the loss WITHOUT the computational graph attached

          preds.extend(pred.cpu().detach().numpy())
          labels.extend(y.cpu().numpy())

    return total_loss / num_batches, mean_absolute_error(labels, preds)

Epoch

In [4]:
def train(model, model_name, num_epochs, train_dataloader, validation_dataloader, loss_fn, optimizer):
  train_history = {'loss': [], 'accuracy': []}
  val_history = {'loss': [], 'accuracy': []}
  best_val_loss = np.inf
  print("Start training...")
  for t in range(num_epochs):
      print(f"\nEpoch {t+1}")
      train_loss, train_acc = epoch_iter(train_dataloader, model, loss_fn, optimizer)
      print(f"Train loss: {train_loss:.3f} \t Train acc: {train_acc:.3f}")
      val_loss, val_acc = epoch_iter(validation_dataloader, model, loss_fn, is_train=False)
      print(f"Val loss: {val_loss:.3f} \t Val acc: {val_acc:.3f}")

      # save model when val loss improves
      if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_dict = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': t}
        torch.save(save_dict, model_name + '_best_model.pth')

      # save latest model
      save_dict = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': t}
      torch.save(save_dict, model_name + '_latest_model.pth')

      # save training history for plotting purposes
      train_history["loss"].append(train_loss)
      train_history["accuracy"].append(train_acc)

      val_history["loss"].append(val_loss)
      val_history["accuracy"].append(val_acc)
      
  print("Finished")
  return train_history, val_history

In [5]:

# Load ResNet model from torchvision (with pretrained=True)
weights = models.ResNet50_Weights.IMAGENET1K_V2
resnet = models.resnet50(weights=weights)
# Freeze all layers except the last one
#for param in resnet.parameters():
#    param.requires_grad = False
transforms = weights.transforms()
# Change the number of neurons in the last layer to the number of classes of the CIFAR10 dataset
# TODO
num_classes = 1
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)


resnet = resnet.to(device)


# Define the model

In [6]:








testDataset = CustomDataset(datasetPath, annotations['splits']['chessred2k']['test']['image_ids'],annotations['annotations']["pieces"],annotations['images'],transforms)
testLoader = DataLoader(testDataset, batch_size=10, shuffle=False, num_workers=0)

trainDataset = CustomDataset(datasetPath, annotations['splits']['chessred2k']['train']['image_ids'],annotations['annotations']["pieces"],annotations['images'],transforms)
trainLoader = DataLoader(trainDataset, batch_size=10, shuffle=False, num_workers=0)
validationDataset = CustomDataset(datasetPath, annotations['splits']['chessred2k']['val']['image_ids'],annotations['annotations']["pieces"],annotations['images'],transforms)
validationLoader = DataLoader(validationDataset, batch_size=10, shuffle=False, num_workers=0)


In [8]:
train(resnet, "resnet50", 50, trainLoader, validationLoader, nn.MSELoss(), torch.optim.Adam(resnet.parameters(), lr=5e-5))

Start training...

Epoch 1


100%|██████████| 145/145 [00:11<00:00, 12.95it/s]


Train loss: 6.651 	 Train acc: 1.979


100%|██████████| 33/33 [00:00<00:00, 40.94it/s]


Val loss: 17.919 	 Val acc: 3.725

Epoch 2


...

100%|██████████| 145/145 [00:10<00:00, 13.58it/s]


Train loss: 0.506 	 Train acc: 0.561


100%|██████████| 33/33 [00:00<00:00, 41.55it/s]


Val loss: 16.460 	 Val acc: 3.571
Finished


({'loss': [6.6507436414217125,
   1.8913418828413404,
   1.0983278893608728,
   0.6498562832844669,
   0.3840759050614875,
   0.22486477206493247,
   0.13057481836656043,
   0.0893536129163514,
   0.08954448015910799,
   0.12575054579125397,
   0.22231028902761896,
   0.23606640462731493,
   0.611378651857376,
   1.2066227317370217,
   1.560108596163577,
   1.2043701273613963,
   1.4072996221482754,
   1.296275490727918,
   0.9370329189467533,
   0.7008833112965884,
   0.6441257041589967,
   0.578833111850866,
   0.6202734758627826,
   0.6827499552565659,
   0.7489048665710565,
   0.6979218454453452,
   0.5530330035321671,
   0.4407086107109127,
   0.40679241468937233,
   0.39106888406235596,
   0.4774621525843596,
   0.5714256770523458,
   0.5089882780251832,
   0.44952424774909844,
   0.44260168142359835,
   0.3657361792400479,
   0.4265965479715117,
   0.5130489885807037,
   0.42691646223438195,
   0.3816845606370219,
   0.41298458988553494,
   0.3944283513544962,
   0.4837625167750